In [9]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
# from importlib import reload
%matplotlib inline
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:80% !important; }</style>")) 

In [3]:
import os
from surprise import SVD, Reader, Dataset, KNNBasic

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

## Load data and merge in Adam's ratings

In [5]:
def load_data():
    cwd = os.getcwd()
    ratings = pd.read_csv(os.path.join(cwd, "..", "data", "ratings.csv"))
    ratings.drop('timestamp', axis=1, inplace=True)
    max_userId = ratings['userId'].max()
    adam_id = max_userId + 1
    adam = pd.read_csv('adam_profile.csv', index_col=0)
    adam['userId'] = adam_id
    adam = adam[['userId', 'movieId', 'rating']]
    ratings_combined = pd.concat([ratings, adam])
    return ratings_combined

In [10]:
ratings_combined = load_data()

## Persist merged data

In [ ]:
cwd = os.getcwd()
path = os.path.join(cwd, "..", "data", "ratings_combined_adam.csv")

In [ ]:
ratings_combined.to_csv(path, index=False)

## Implement Surprise KNN algorithm

In [4]:
cwd = os.getcwd()
path = os.path.join(cwd, "..", "data", "ratings_combined_adam.csv")

In [5]:
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)

In [6]:
data = Dataset.load_from_file(path, reader=reader)

In [7]:
trainset = data.build_full_trainset()

In [10]:
sim_options = {'user_based':False}  # user_based = True causes a MemoryError at the training step

In [11]:
algo = KNNBasic(sim_options=sim_options)

In [12]:
algo.train(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [20]:
iids_inner = trainset.all_items()

In [36]:
iids_inner

xrange(26744)

In [21]:
iids_raw = [trainset.to_raw_iid(id) for id in iids_inner]

In [37]:
iids_raw[:10]

['2', '29', '32', '47', '50', '112', '151', '223', '253', '260']

In [39]:
uids_raw = [trainset.to_raw_uid(id) for id in trainset.all_users()]

In [42]:
max(uids_raw)

'99999'

In [47]:
trainset.to_raw_uid('99998')

ValueError: 99998 is not a valid inner id.

In [38]:
trainset.all_users()

xrange(138494)

In [28]:
uid = max(trainset.all_users()); uid

138493

In [30]:
preds = algo.predict(uid, iids)

3.5255474322998368